# Demostration of the recommenders

In [1]:
import pandas as pd
import numpy as np
from surprise import dump
from evaluation.Evaluate import Evaluate, Evaluate_CFR

from model.CollaborativeFilteringRec import train_cf_models
import model.CollaborativeFilteringRec as CFR
import model.HybridRecommendationSystem as HR
import model.ContentBasedRec as CBR

movie_df = pd.read_csv('./data/external/movies.csv')

## Preprocessing
* Preprocessed data for both content-based and colloborative filtering systems.
* Updates the precalculated dataframes.

In [4]:
print("Prerocessing for Collaborative Filtering Recommender...")
from data_script.preprocess_collaborative import sveta_user_id, hamid_user_id
# import clustering_collaborative #TODO

Prerocessing for Collaborative Filtering Recommender...
Creating raw dataframes...
Creating dataframes with Sveta and Hamid ratings...
Creating dataframes with anti-Sveta and anti-Hamid ratings...
Creating new ratings dataframe for collaborative filtering ...
Out of total of  138493  users,  64051  are considered lazy and will be removed.
The final number of users is  74442
Randomly choosing 1000 users...
Adding Sviatlana's and Hamid's ratings... we are lazy users, but still...
Sveta 's user id is 62095
Hamid 's user id is 11888
Adding anti-Sviatlana's and anti-Hamid's ratings... they are lazy users, but still...
anti-Sveta 's user id is 18131
anti-Hamid 's user id is 38052
Now, the final number of users is  1004
Filtering movies...
Out of total of  9328  movies,  8803  are considered rare and will be removed.
The final number of movies is  525
Creating the pivot matrix...
The resulting sparcity of the matrix is: 0.7993208119901347
Preparing the final rating matrix...
Data preprocesssi

In [9]:
# pick a user
ratings = pd.read_csv('./data/processed/final_ratings.csv')
random_user = np.random.choice(ratings.userId.unique()) # picks a random user
sveta_user_id, hamid_user_id # or use sveta and hamid's id (assigned randomly at the preprocessing step

(62095, 11888)

In [5]:
print("Prerocessing for Content-Based Recommender...")
from data_script.preprocess_content_based import get_df_ContBaseRec
df_ContBaseRec = get_df_ContBaseRec()

Prerocessing for Content-Based Recommender...


/home/mumu/Documents/DS/movie_recommendation_ProjectCDS/data_script/preprocess_content_based.py:93: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_movies.genres.apply(pd.Series).stack()).sum(level=0)


## Train Nearest Neighbor CB model

In [2]:
nn = CBR.train_nearest_neighbors_model()

## Training and saving CF models

In [8]:
trainset, testset, data, algo, algo_predictions, knn, knn_predictions = train_cf_models()

Getting the ratings matrix...
Preparing data in the Suprise format...
Training and saving a KNN model for calculating the CF model weights in hybrid setup..
Mean CV RMSE is 0.837
Training KNN-based memory based model for hybrid...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Mean CV RMSE is 0.933
Training is done!


## Content-based recommender

In [4]:
cb_df = CBR.recommendation(sveta_user_id, 20)
cb_df.head(20)

NameError: name 'sveta_user_id' is not defined

## User-based colloborative filtering recommender

In [11]:
cf_model = CFR.CollaborativeFilteringRecommender()
cf_model.fit_and_predict()
cf_model.recommend(sveta_user_id, 20).merge(movie_df, on='movieId', how='left')

Getting the ratings matrix...
Preparing data in the Suprise format...
Done calculating predictions and scores!


,userId,movieId,rating,cf_score,title,genres
0,62095,32,4.796649,0.954811,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
1,62095,912,4.692394,0.931643,Casablanca (1942),Drama|Romance
2,62095,1230,4.580400,0.906756,Annie Hall (1977),Comedy|Romance
3,62095,1219,4.559330,0.902073,Psycho (1960),Crime|Horror
4,62095,1233,4.499145,0.888699,"Boot, Das (Boat, The) (1981)",Action|Drama|War
5,62095,904,4.461464,0.880325,Rear Window (1954),Mystery|Thriller
6,62095,1204,4.444006,0.876446,Lawrence of Arabia (1962),Adventure|Drama|War
7,62095,5995,4.434802,0.874401,"Pianist, The (2002)",Drama|War
8,62095,1288,4.430102,0.873356,This Is Spinal Tap (1984),Comedy
9,62095,1207,4.423082,0.871796,To Kill a Mockingbird (1962),Drama


## Hybrid recommender

In [12]:
h_df = HR.hybrid_recommendation(sveta_user_id)
h_df.head(20)

Getting the ratings matrix...
Preparing data in the Suprise format...
Done calculating predictions and scores!
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.


,movieId,title,genres,hybrid_score
36,2692,Run Lola Run (Lola rennt) (1998),Action|Crime,1.057184
1,2858,American Beauty (1999),Comedy|Drama,1.041630
19,1201,"Good, the Bad and the Ugly, The (Buono, il bru...",Action|Adventure|Western,1.033640
9,1213,Goodfellas (1990),Crime|Drama,1.032253
29,1276,Cool Hand Luke (1967),Drama,1.030731
7,2329,American History X (1998),Crime|Drama,1.025776
77,1358,Sling Blade (1996),Drama,1.023372
120,2762,"Sixth Sense, The (1999)",Drama|Horror|Mystery,1.019424
46,1266,Unforgiven (1992),Drama|Western,1.018515
17,1304,Butch Cassidy and the Sundance Kid (1969),Action|Western,1.015850


# Evaluation of the different recomenders


## Accuracy of the collaborative system
* metrics: rmse
* method: cross-validation


In [14]:
cf_model.cross_validate()

Cross Validating the data...
Mean CV RMSE is 0.836


0.836

## Variety
The measure represents the averaged percentage of all the movies recommended to all the users (1 corresponds to 100%).

In [2]:
Evaluate = Evaluate()

Getting the ratings matrix...
Preparing data in the Suprise format...
Done calculating predictions and scores!


In [5]:
variety_cf = Evaluate.variety_collaborative_filtering()
print("Collaborative filtering:", variety_cf)

Collaborative filtering: (0.151, 0.0)


In [3]:
variety_cb = Evaluate.variety_content_based() # as CB is unoptimized, it takes a really long time
print("Content-based:", variety_cb)

Content-based: (0.148, 0.0)


In [3]:
variety_hybrid = Evaluate.variety_hybrid() # as CB is unoptimized, it takes a really long time...
print("Hybrid:", variety_hybrid)

Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done 

## Personalization

# creating CB recommendation Matrix with predicted score per movie for all the Users present in data\processed\final_rating.csv

In [1]:
from model.ContentBasedRec import get_CBMatrix 
df = get_CBMatrix()

KeyboardInterrupt: 

# creating HR recommendation Matrix with predicted score per movie for all the Users present in data\processed\final_rating.csv

In [1]:
from model.HybridRecommendationSystem import get_HRMatrix
df = get_HRMatrix()

Getting the ratings matrix...
Preparing data in the Suprise format...
Done calculating predictions and scores!
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
user nr. 15614
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
user nr. 130835
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
user nr. 37477
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
user nr. 34608
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing the pearson similarity matrix...
Done computing similarity matrix.
user nr. 69
Getting the ratings matrix...
Preparing data in the Suprise format...
Computing t

KeyError: 'hybrid_score'